In [ ]:
!sudo apt install libspatialindex-dev

[sudo] password for harry: 

In [ ]:
!wget https://github.com/Inder-Dhillon/Toronto-Accidents-As-A-Network/blob/main/accident.py?raw=true
!mv accident.py\?raw\=true accident.py

In [ ]:
pip install osmnx

In [ ]:
import osmnx as ox
import networkx as nx
import pandas as pd
import json
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt
from accident import ACCIDENT

In [ ]:
G = ox.graph_from_place('Toronto, Ontario, Canada', network_type='drive')

In [ ]:
#G = ox.consolidate_intersections(ox.project_graph(G), tolerance=30, rebuild_graph=True)

In [ ]:
ox.plot_graph(G, bgcolor='white', node_size=0.3, node_color="red", edge_linewidth=0.4, edge_alpha=1, dpi=300, figsize=(10,10))

In [8]:
print(nx.info(G))

Name: 
Type: MultiDiGraph
Number of nodes: 27440
Number of edges: 73176
Average in degree:   2.6668
Average out degree:   2.6668


In [9]:
edges =  list(G.edges.data())
edges[160]

(3463503,
 3463505,
 {'geometry': <shapely.geometry.linestring.LineString at 0x7f5c31d326d8>,
  'highway': 'secondary',
  'lanes': '4',
  'length': 119.022,
  'maxspeed': '50',
  'name': 'Lawrence Avenue East',
  'oneway': False,
  'osmid': 163720386})

In [10]:
nodes =  list(G.nodes.data())
nodes[1]

(1497798, {'osmid': 1497798, 'x': -79.3355379, 'y': 43.7580721})

In [11]:
nx.number_of_selfloops(G)

762

## KSI Dataset:

In [12]:
ksi_df = gpd.read_file("https://opendata.arcgis.com/datasets/cc17cc27ee5a4989b78d9a3810c6c007_0.geojson")

In [13]:
ksi_df.head()

,Index_,ACCNUM,YEAR,DATE,TIME,HOUR,STREET1,STREET2,OFFSET,ROAD_CLASS,District,WardNum,Division,LATITUDE,LONGITUDE,LOCCOORD,ACCLOC,TRAFFCTL,VISIBILITY,LIGHT,RDSFCOND,ACCLASS,IMPACTYPE,INVTYPE,INVAGE,INJURY,FATAL_NO,INITDIR,VEHTYPE,MANOEUVER,DRIVACT,DRIVCOND,PEDTYPE,PEDACT,PEDCOND,CYCLISTYPE,CYCACT,CYCCOND,PEDESTRIAN,CYCLIST,AUTOMOBILE,MOTORCYCLE,TRUCK,TRSN_CITY_VEH,EMERG_VEH,PASSENGER,SPEEDING,AG_DRIV,REDLIGHT,ALCOHOL,DISABILITY,Hood_ID,Neighbourhood,ObjectId,geometry
0,3389067,893184,2006,2006-01-01T05:00:00+00:00,236,2,WOODBINE AVE,O CONNOR DR,None,Major Arterial,Toronto and East York,19.0,54,43.699595,-79.318797,Intersection,Intersection Related,No Control,Clear,Dark,Wet,Non-Fatal Injury,Approaching,Passenger,50 to 54,Major,None,None,None,None,None,None,None,None,None,None,None,None,None,None,Yes,None,None,None,None,Yes,Yes,Yes,None,Yes,None,60,Woodbine-Lumsden (60),1,POINT (-79.31880 43.69960)
1,3389068,893184,2006,2006-01-01T05:00:00+00:00,236,2,WOODBINE AVE,O CONNOR DR,None,Major Arterial,Toronto and East York,19.0,54,43.699595,-79.318797,Intersection,Intersection Related,No Control,Clear,Dark,Wet,Non-Fatal Injury,Approaching,Passenger,15 to 19,Minor,None,None,None,None,None,None,None,None,None,None,None,None,None,None,Yes,None,None,None,None,Yes,Yes,Yes,None,Yes,None,60,Woodbine-Lumsden (60),2,POINT (-79.31880 43.69960)
2,3389069,893184,2006,2006-01-01T05:00:00+00:00,236,2,WOODBINE AVE,O CONNOR DR,None,Major Arterial,Toronto and East York,19.0,54,43.699595,-79.318797,Intersection,Intersection Related,No Control,Clear,Dark,Wet,Non-Fatal Injury,Approaching,Driver,55 to 59,Minor,None,North,"Automobile, Station Wagon",Going Ahead,Driving Properly,Normal,None,None,None,None,None,None,None,None,Yes,None,None,None,None,Yes,Yes,Yes,None,Yes,None,60,Woodbine-Lumsden (60),3,POINT (-79.31880 43.69960)
3,3389070,893184,2006,2006-01-01T05:00:00+00:00,236,2,WOODBINE AVE,O CONNOR DR,None,Major Arterial,Toronto and East York,19.0,54,43.699595,-79.318797,Intersection,Intersection Related,No Control,Clear,Dark,Wet,Non-Fatal Injury,Approaching,Passenger,20 to 24,Minor,None,None,None,None,None,None,None,None,None,None,None,None,None,None,Yes,None,None,None,None,Yes,Yes,Yes,None,Yes,None,60,Woodbine-Lumsden (60),4,POINT (-79.31880 43.69960)
4,3389071,893184,2006,2006-01-01T05:00:00+00:00,236,2,WOODBINE AVE,O CONNOR DR,None,Major Arterial,Toronto and East York,19.0,54,43.699595,-79.318797,Intersection,Intersection Related,No Control,Clear,Dark,Wet,Non-Fatal Injury,Approaching,Passenger,15 to 19,Minor,None,None,None,None,None,None,None,None,None,None,None,None,None,None,Yes,None,None,None,None,Yes,Yes,Yes,None,Yes,None,60,Woodbine-Lumsden (60),5,POINT (-79.31880 43.69960)


In [14]:
def assign_injury_index(injury:str):
  if injury == "Minimal":
    return 0.4
  elif injury == "Minor":
    return 0.6
  elif injury == "Major":
    return 0.8
  elif injury == "Fatal":
    return 1.0
  else:
    return 0.2

In [15]:
ksi_df["INJ_INDEX"] = ksi_df["INJURY"].apply(lambda x: assign_injury_index(x))

In [16]:
index_df = ksi_df[["ACCNUM", "INJ_INDEX"]].groupby(by="ACCNUM").sum()

Combining Date and Hour to have one single column:

In [17]:
ksi_df["DATE"] = pd.to_datetime(ksi_df['DATE']).dt.floor("D") + pd.to_timedelta(ksi_df["HOUR"], unit="hour")

Add datapoints to keep here:

In [18]:
cols_to_keep = ["LATITUDE", "LONGITUDE", "ACCNUM", "DATE", "VISIBILITY", "LIGHT",	"RDSFCOND"]
ksi_df = ksi_df[cols_to_keep]

Counting fatalities using duplicate rows:

In [19]:
fatalities = ksi_df["ACCNUM"].value_counts()
ksi_df["FATALITIES"] = ksi_df["ACCNUM"].apply(lambda x : fatalities[x])

In [20]:
ksi_df = ksi_df.drop_duplicates()
ksi_df.reset_index(drop=True, inplace=True)

In [21]:
ksi_df["G_NODE"]= ox.get_nearest_nodes(G, ksi_df["LONGITUDE"], ksi_df["LATITUDE"], method="balltree")
ksi_df.drop(["LATITUDE", "LONGITUDE"], axis=1, inplace=True)

In [22]:
ksi_df = ksi_df.merge(index_df, on="ACCNUM")

In [23]:
ksi_df = ksi_df.infer_objects()
ksi_df

,ACCNUM,DATE,VISIBILITY,LIGHT,RDSFCOND,FATALITIES,G_NODE,INJ_INDEX
0,893184,2006-01-01 02:00:00+00:00,Clear,Dark,Wet,8,33771082,4.2
1,909646,2006-01-01 03:00:00+00:00,Snow,"Dark, artificial",Slush,5,158754118,2.4
2,884090,2006-01-02 07:00:00+00:00,Other,"Dark, artificial",Wet,2,21631723,1.0
3,885782,2006-01-04 19:00:00+00:00,Rain,Dark,Wet,2,26007459,1.0
4,882079,2006-01-06 22:00:00+00:00,Clear,"Dark, artificial",Dry,2,259750208,1.0
...,...,...,...,...,...,...,...,...
5685,9002468442,2019-12-23 06:00:00+00:00,Clear,Dawn,Wet,2,86186677,1.0
5686,9002471073,2019-12-23 14:00:00+00:00,Clear,Daylight,Dry,3,287946845,1.8
5687,9002481518,2019-12-25 08:00:00+00:00,Clear,Daylight,Wet,2,1498033,1.0
5688,9002496688,2019-12-27 23:00:00+00:00,Clear,"Dark, artificial",Dry,2,392516842,1.0


Creating column of accident objects:

In [24]:
ksi_df["ACCIDENT"] = ksi_df.apply(lambda x: ACCIDENT(x["ACCNUM"], x["DATE"], x["VISIBILITY"], x["LIGHT"], x["RDSFCOND"], x["FATALITIES"], x["INJ_INDEX"]), axis=1)

In [25]:
ksi_df

,ACCNUM,DATE,VISIBILITY,LIGHT,RDSFCOND,FATALITIES,G_NODE,INJ_INDEX,ACCIDENT
0,893184,2006-01-01 02:00:00+00:00,Clear,Dark,Wet,8,33771082,4.2,"ACCIDENT(acc_id='893184', date=Timestamp('2006..."
1,909646,2006-01-01 03:00:00+00:00,Snow,"Dark, artificial",Slush,5,158754118,2.4,"ACCIDENT(acc_id='909646', date=Timestamp('2006..."
2,884090,2006-01-02 07:00:00+00:00,Other,"Dark, artificial",Wet,2,21631723,1.0,"ACCIDENT(acc_id='884090', date=Timestamp('2006..."
3,885782,2006-01-04 19:00:00+00:00,Rain,Dark,Wet,2,26007459,1.0,"ACCIDENT(acc_id='885782', date=Timestamp('2006..."
4,882079,2006-01-06 22:00:00+00:00,Clear,"Dark, artificial",Dry,2,259750208,1.0,"ACCIDENT(acc_id='882079', date=Timestamp('2006..."
...,...,...,...,...,...,...,...,...,...
5685,9002468442,2019-12-23 06:00:00+00:00,Clear,Dawn,Wet,2,86186677,1.0,"ACCIDENT(acc_id='9002468442', date=Timestamp('..."
5686,9002471073,2019-12-23 14:00:00+00:00,Clear,Daylight,Dry,3,287946845,1.8,"ACCIDENT(acc_id='9002471073', date=Timestamp('..."
5687,9002481518,2019-12-25 08:00:00+00:00,Clear,Daylight,Wet,2,1498033,1.0,"ACCIDENT(acc_id='9002481518', date=Timestamp('..."
5688,9002496688,2019-12-27 23:00:00+00:00,Clear,"Dark, artificial",Dry,2,392516842,1.0,"ACCIDENT(acc_id='9002496688', date=Timestamp('..."


Collapsing dataset to get list of accidents for each node:

In [26]:
ksi_df = ksi_df.groupby('G_NODE')['ACCIDENT'].apply(list).reset_index(name='ACCIDENTS')

In [27]:
ksi_df

,G_NODE,ACCIDENTS
0,1497983,"[ACCIDENT(acc_id='7000804511', date=Timestamp(..."
1,1498018,"[ACCIDENT(acc_id='1126375', date=Timestamp('20..."
2,1498033,"[ACCIDENT(acc_id='1152152', date=Timestamp('20..."
3,1498042,"[ACCIDENT(acc_id='1298032', date=Timestamp('20..."
4,1498116,"[ACCIDENT(acc_id='7001054440', date=Timestamp(..."
...,...,...
3704,7620170933,"[ACCIDENT(acc_id='1025433', date=Timestamp('20..."
3705,7870612785,"[ACCIDENT(acc_id='7002059893', date=Timestamp(..."
3706,7870612852,"[ACCIDENT(acc_id='922700', date=Timestamp('200..."
3707,7891518884,"[ACCIDENT(acc_id='143084786', date=Timestamp('..."


Converting dataset to dict to set node attributes:

In [28]:
attr = ksi_df.set_index('G_NODE')['ACCIDENTS'].to_dict()

In [29]:
nx.set_node_attributes(G, [], "accident_list")
nx.set_node_attributes(G, attr, "accident_list")

Example Nodes:

In [30]:
G.nodes[1497793]

{'accident_list': [],
 'highway': 'motorway_junction',
 'osmid': 1497793,
 'ref': '14',
 'x': -79.3346901,
 'y': 43.7555309}

In [31]:
G.nodes[158754118]

{'accident_list': [ACCIDENT(acc_id='909646', date=Timestamp('2006-01-01 03:00:00+0000', tz='UTC'), visibility='Snow', light='Dark, artificial', road_conditions='Slush', fatalities=5, inj_index=2.4000000000000004)],
 'osmid': 158754118,
 'x': -79.3162254,
 'y': 43.6849494}